### hadoop ecosystem

1. hadoop

1. mysql

1. hive

1. hbase

1. zookeeper

1. sqoop

1. Q & A

### install hadoop

\$ brew install hadoop

/usr/local/Cellar/hadoop/2.7.0

### config hadoop-env.sh

\$ pico libexec/etc/hadoop/hadoop-env.sh

replace

export HADOOP_OPTS="$HADOOP_OPTS -Djava.net.preferIPv4Stack=true"

with

export HADOOP_OPTS="$HADOOP_OPTS -Djava.net.preferIPv4Stack=true -Djava.security.krb5.realm= -Djava.security.krb5.kdc="

### config mapred-site.xml

\$ pico libexec/etc/hadoop/mapred-site.xml

    <configuration>
        <property>
            <name>mapred.job.tracker</name>
            <value>localhost:9010</value>
        </property>
    </configuration>

### config hdfs-site.xml

\$ pico libexec/etc/hadoop/hdfs-site.xml

    <configuration>
        <property>
            <name>dfs.replication</name>
            <value>1</value>
        </property>
    </configuration>

### config HADOOP_HOME and alias 

\$ pico ~/.bash_profile

export HADOOP_HOME=/usr/local/Cellar/hadoop/2.7.0

alias hstart="/usr/local/Cellar/hadoop/2.7.0/sbin/start-dfs.sh;/usr/local/Cellar/hadoop/2.7.0/sbin/start-yarn.sh"

alias hstop="/usr/local/Cellar/hadoop/2.7.0/sbin/stop-yarn.sh;/usr/local/Cellar/hadoop/2.7.0/sbin/stop-dfs.sh"

\$ source ~/.bash_profile

### config ssh

generate rsa keys

\$ ssh-keygen -t rsa

`enable remote login`

system preferences -> sharing -> remote login

`authorize ssh keys`

$ cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys

`login via ssh`

$ ssh localhost

### format start stop

\$ hdfs namenode -format

\$ hstart

\$ jps

\$ hadoop jar path/to/hadoop-examples.jar pi 10 100

\$ hdfs dfs -mkdir -p /user/hqlgree2

\$ hdfs dfs -put book.txt /user/hqlgree2/

\$ hstop

### install mysql

\$ brew install mysql

start mysql

\$ mysql.server start

stop mysql

\$ mysql.server stop

### install hive

\$ brew install hive

/usr/local/Cellar/hive/1.0.0

### config ~/.bash_profile

\$ pico ~/.bash_profile

export HIVE_HOME=/usr/local/Cellar/hive/1.1.0/libexec

export HCAT_HOME=/usr/local/Cellar/hive/1.1.0/libexec/hcatalog

\$ source ~/.bash_profile

### config mysql connector

\$ curl -L 'http://www.mysql.com/get/Downloads/Connector-J/mysql-connector-java-5.1.22.tar.gz/from/http://mysql.he.net/' | tar xz

\$ cd mysql-connector-java-5.1.22

\$ sudo cp mysql-connector-java-5.1.22-bin.jar /usr/local/Cellar/hive/1.1.0/libexec/lib/

### config metastore

\$ mysql -u root

mysql> create database metastore;

mysql> use metastore;

mysql> create use 'hiveuser'@'localhost' identified by 'password';

mysql> grant select, insert, update, delete, alter, create, index on metastore.* to 'hiveuser';

### config hive-site.xml

\$ cd libexec/conf

\$ cp hive-default.xml.template hive-site.xml

    <property>
        <name>javax.jdo.option.ConnectionURL</name>
        <value>jdbc:mysql://localhost/metastore</value>
    </property>
    <property>
        <name>javax.jdo.option.ConnectionDriverName</name>
        <value>com.mysql.jdbc.Driver</value>
    </property>
    <property>
        <name>javax.jdo.option.ConnectionUserName</name>
        <value>hiveuser</value>
    </property>
    <property>
        <name>javax.jdo.option.ConnectionPassword</name>
        <value>password</value>
    </property>
    <property>
        <name>datanucleus.fixedDatastore</name>
        <value>false</value>
    </property>

### config hive-site.xml

replace
    
    ${system:java.io.tmpdir}/${system:user.name} => /tmp/mydir
    
with

    ${system:java.io.tmpdir}/${hive.session.id} => /usr/local/Cellar/hive/1.0.0/libexec/iotmp

### test hive

\$ hive

hive> show tables;

hive> create table temp_table temp_col string;

### troubleshooting

\$ mysql -u root

mysql> set global binlog_format = 'ROW';

\$ hive -hiveconf hive.root.logger=INFO,console

read raw hive log located at /tmp/hqlgree2/hive.log

### install hbase

\$ brew install hbase

/usr/local/Cellar/hbase/1.0.1

if you got 404

run brew update first

\$ brew update

### install zookeeper

\$ brew install zookeeper

/usr/local/Cellar/zookeeper/3.4.6_1

### config zookeeper

\$ mkdir /usr/local/zookeeper/data

\$ chown -R host1 /usr/local/zookeeper

\$ pico zoo.cfg

tickTime=2000

initLimit=10

syncLimit=5

dataDir=/usr/local/zookeeper/data

clientPort=2181

\$ pico ~/.bash_profile

export ZOOKEEPER_HOME="/usr/local/Cellar/zookeeper/3.4.6_1"

### start, status, client demo, stop

\$ zkServer.sh start

\$ zkServer.sh status

\$ zkCli.sh -server localhost:2181

[zk:...] ls

[zk:...] create /my_app /some_data

[zk:...] get /my_app

[zk:...] set /my_app /new_data

[zk:...] delete /my_app

\$ skServer.sh stop

### config zookeeper - 3 nodes cluster

<pre>
+-------+   +-------+   +-------+
|       |   |       |   |       |
| host1 |   | host2 |   | host3 |
|       |   |       |   |       |
+-------+   +-------+   +-------+
</pre>

after configuration

login all these 3 boxes

start

\$ zkServer.sh start

### config zookeeper - 3 nodes cluster zoo.cfg

\$ pico zoo.cfg

tickTime=2000

initLimit=10

syncLimit=5

dataDir=/usr/local/zookeeper/data

clientPort=2181

server.1=host1:20010:20011

server.2=host2:20010:20011

server.3=host2:20010:20011

### config zookeeper - 3 nodes cluster

\$ scp zoo.cfg hqlgree2@host1:/usr/local/Cellar/zookeeper/3.4.6_1/

\$ scp zoo.cfg hqlgree2@host2:/usr/local/Cellar/zookeeper/3.4.6_1/

\$ scp zoo.cfg hqlgree2@host3:/usr/local/Cellar/zookeeper/3.4.6_1/


\$ echo 1 myid

\$ scp myid hqlgree2@host1:/usr/local/zookeeper/data/

\$ echo 2 myid

\$ scp myid hqlgree2@host2:/usr/local/zookeeper/data/

\$ echo 3 myid

\$ scp myid hqlgree2@host3:/usr/local/zookeeper/data/

### install sqoop

\$ brew install sqoop

/usr/local/Cellar/sqoop/1.4.5

### five minutes demo

\$ sqoop2-shell

sqoop:000> set server --host localhost --port 12000 --webapp sqoop

sqoop:000> show version --all

sqoop:000> show connector

sqoop:000> create link -c 2 # generic jdbc connector

sqoop:000> create link -c 1 # hdfs connector

sqoop:000> show link --all

sqoop:000> create job -f 1 -t 2

sqoop:000> start job -j 1

sqoop:000> status job -j 1

sqoop:000> start job -j 1 -s

sqoop:000> stop job -j 1

### mysql hdfs demo - create database and table

$ mysql -u root -p

mysql> create database sqoop;

mysql> use sqoop;

mysql> create table employee(

->  emp_id int not null auto_increment,

->  emp_name  varchar(32) not null,

->  emp_title varchar(32) not null,

->  primary key ( emp_id )

->);

### mysql hdfs demo - test data

mysql> insert into employee values (1, 'name1', 'dev'), (2, 'name2', 'pm'),

-> (3, 'name3', 'qa'), (4, 'name4', 'test'), (5, 'name5', 'dev');

Query OK, 5 rows affected (0.00 sec)

Records: 5  Duplicates: 0  Warnings: 0

mysql> select * from employee;

<pre>
+--------+----------+-----------+
| emp_id | emp_name | emp_title |
+--------+----------+-----------+
|      1 | name1    | dev       |
|      2 | name2    | pm        |
|      3 | name3    | qa        |
|      4 | name4    | test      |
|      5 | name5    | dev       |
+--------+----------+-----------+
</pre>

5 rows in set (0.00 sec)

### mysql hdfs demo - import

\$ sqoop import --connect jdbc:mysql://localhost/sqoop \

--username sqoop --password sqoop --table employee \

--target-dir /usr/hqlgree2/sqoopdata/employee

\$ sqoop import --connect jdbc:mysql://localhost/sqoop \

--username sqoop --password sqoop --table employee \

--target-dir sqoopdata/employee

\$ sqoop import --connect jdbc:mysql://localhost/sqoop \

--username sqoop --password sqoop --table employee \

--warehouse-dir sqoopdata -m 1 \

--check-column emp_id \

--incremental append \

--last-value 5

### mysql hdfs demo - export

\$ sqoop export --connect jdbc:mysql://localhost/test \

--username sqoop -P --table employee -m 1 \

--export-dir '/usr/hqlgree2/sqoopdata/employee/part-m-00000'

\$ sqoop --options-file export.txt --table employee \

--export-dir '/usr/hqlgree2/sqoopdata/employee/part-m-00001' \

--update-key emp_id

### sqlserver hdfs demo - config sqljdbc

\$ curl -L 'http://download.microsoft.com/download/0/2/A/02AAE597-3865-456C-AE7F-613F99F850A8/sqljdbc_4.0.2206.100_enu.tar.gz' | tar xz

\$ cd sqljdbc_4.0/enu

\$ cp sqljdbc4.jar /usr/local/Cellar/sqoop/1.4.5/libexec/lib/

\$ hdfs dfs -put sqljdbc4.jar /usr/local/Cellar/sqoop/1.4.5/libexec/lib/

### sqlserver hdfs demo - codegen

\$ sqoop codegen --connect 'jdbc:sqlserver://192.168.100.100;database=sqoop;username=sa;password=as' --table employee

INFO orm.CompilationManager: Writing jar file: /tmp/sqoop-hqlgree2/compile/38c5c07622a71bc6695d397b2ad36e40/employee.jar

\$ hdfs dfs -mkdir /tmp/sqoop-hqlgree2/compile/38c5c07622a71bc6695d397b2ad36e40/

\$ hdfs dfs -put /tmp/sqoop-hqlgree2/compile/38c5c07622a71bc6695d397b2ad36e40/* \

/tmp/sqoop-hqlgree2/compile/38c5c07622a71bc6695d397b2ad36e40/

\$ hdfs dfs -rm -r /user/hqlgree2/mssql/employee

### sqlserver demo - import.mssql

\$ pico import.mssql

import

-libjars

/tmp/sqoop-hqlgree2/compile/38c5c07622a71bc6695d397b2ad36e40/employee.jar

--connect

'jdbc:sqlserver://192.168.100.100;database=sqoop;username=sa;password=as'

### sqlserver hdfs demo - import export

\$ sqoop --options-file mssql.options --table employee --target-dir mssql/employee


\$ sqoop export --connect jdbc:sqlserver://192.168.100.100:1433;database=sqoop \

--username sa --password as --table movie \

--export-dir /user/hqlgree2/sqoopdata/employee/part-m-00000 \

--import-fields-terminated-by '\t'

# Thanks

Q & A